<a href="https://colab.research.google.com/github/ReemALHamed/Creating-chatbot-model/blob/main/ReemzChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tflearn 

     |████████████████████████████████| 107 kB 23.9 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=f6ebe03b5c83277b657693f0da0dac5f88ac57474ee640a5e0701a24b529eaf4
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [2]:
#libraries needed for NLP
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

#Libararies needed for TensorFlow processing
from tensorflow.python.framework import ops
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
# import the Json file containing the chat-bot intents
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up" , "Good morning" , "Good evening" , "hello" , "hey" , "what\'s up"],\r\n         "responses": ["Hello!", "Good to see you!", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day","bye","i have to go","gotta go"],\r\n         "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "age",\r\n         "patterns": ["how old", "how old are you", "what is your age", "how old are you", "age?"],\r\n         "responses": ["I am old and wise", "trying to stay fit and young!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "name",\r\n         "patterns": ["what is your name", "what should I call you", 

In [5]:
#import chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [6]:
# print the i=Json file content
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up',
    'Good morning',
    'Good evening',
    'hello',
    'hey',
    "what's up"],
   'responses': ['Hello!', 'Good to see you!', 'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day',
    'bye',
    'i have to go',
    'gotta go'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'tag': 'goodbye'},
  {'context_set': '',
   'patterns': ['how old',
    'how old are you',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am old and wise', 'trying to stay fit and young!'],
   'tag': 'age'},
  {'context_set': '',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?',
    'who are you?'],
   'responses': ['You can call me Reemz.', "I'm Re

In [7]:
words = []
classes = []
documents = []
ignore = ['?']

#loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each and every word in the sentence
        w=nltk.word_tokenize(pattern)
        
        #add word to the word list
        words.extend(w)
        
        #add word(s) to documents
        documents.append((w, intent['tag']))
        
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
#perform stemming and lower each word as well as remove duplicates

words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

#remove duplicate classes
classes=sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

32 documents
5 classes ['age', 'goodbye', 'greeting', 'help', 'name']
46 unique stemmed words ["'s", 'a', 'ag', 'am', 'anyon', 'ar', 'ask', 'bye', 'cal', 'can', 'cya', 'day', 'do', 'ev', 'go', 'good', 'goodby', 'got', 'hav', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'me', 'morn', 'nam', 'old', 'see', 'should', 'someth', 'ta', 'tel', 'ther', 'to', 'up', 'what', 'who', 'yo', 'you']


In [9]:
#create training data
training=[]
output=[]

#create an empty array for output
output_empty=[0]*len(classes)

#create training set bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag=[]
  #list of tokenized words for the pattern
  pattern_words=doc[0]
  #stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
  #output is 1 for current tag and o for the rest of other tags
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
    
  training.append([bag,output_row])
    
#shuffling features and turning it into np.array
random.shuffle(training)
training=np.array(training)
  
#creating training lists
train_x =list(training[:,0])
train_y =list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
#resetting underlying graph data
ops.reset_default_graph()

#building neural network
net=tflearn.input_data(shape=[None, len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]), activation='softmax')
net=tflearn.regression(net)

#defining model and setting up tensorboard
model=tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00726 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.00726 - acc: 1.0000 -- iter: 24/32
Training Step: 4000  | total loss: 0.00731 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.00731 - acc: 1.0000 -- iter: 32/32
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
import pickle
pickle.dump({'word':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data","wb"))

In [12]:
#restoring all the data structures
data = pickle.load(open("training_data","rb"))

words   =data['word']
classes =data['classes']
train_x =data['train_x']
train_y =data['train_y']

In [13]:
with open('intents.json') as json_data:
    intents = json.load(json_data)
  

In [14]:
#load the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [15]:
def clean_up_sentence(sentence):
  #tokenize the token
  sentence_words=nltk.word_tokenize(sentence)
  #stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

#return bag of word array:0 and 1 for each word in the bag that exists
def bow(sentence, words, show_details=False):
  #tokenize sentence
  sentence_words=clean_up_sentence(sentence)
  #generating bag of words
  bag=[0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
        
        if show_details:
          print("found in bag: %s" % w)
  return (np.array(bag))

In [16]:
ERROR_THRESHOLD = 0.0
def classify(sentence):
  #generate probabilities from the model
  results = model.predict([bow(sentence, words)])[0]
  
  #filter out prediction below a threshold
  results = [[i,r] for i, r in enumerate(results) if r> ERROR_THRESHOLD]
  
  #sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  
  #return tuple of intent and probability
  return return_list

def response(sentence, userID='123', show_details=False):
  results=classify(sentence)
  
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)
  

In [17]:
classify('how old are you?')

[('age', 0.99053675),
 ('greeting', 0.006074506),
 ('help', 0.0032399264),
 ('name', 0.00011791937),
 ('goodbye', 3.0966672e-05)]

In [18]:
response('how old are you?')

I am old and wise


In [19]:
response('Bye')

Sad to see you go :(


In [20]:
response('good day')

Hello!


In [21]:
response('what is your name?')

I'm Reemz.


In [23]:
msg=''        
### MAIN PROGRAM ###
print("\n Hey ! I'm a Chatbot !\n Start a Conversation with me ! :D\n\n\n")

while classify(msg)[0][0] != 'goodbye':    
    # get the user input
    msg = input("You: ")
    print("Bot:",end=' ')
    response(msg)


 Hey ! I'm a Chatbot !
 Start a Conversation with me ! :D



You: hi
Bot: Hi there, how can I help?
You: how old are you?
Bot: I am old and wise
You: what is your name?
Bot: You can call me Reemz.
You: what do you do?
Bot: I can help you with whatever you tell me to.
You: bye 
Bot: Sad to see you go :(
